In [1]:
import sys
sys.path.append("../")
import re
from os.path import join
from IPython.display import display
import gc
from collections.abc import Iterable


import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('text', usetex=False)

import matplotlib.pyplot as plt
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import faiss

from lolip.models.torch_utils.archs import *
from lolip.variables import auto_var
from params import *
from utils import params_to_dataframe
from notebook_utils import get_model_name, get_arch, get_ds_name

fontsize=15
%load_ext autoreload
%autoreload 2

In [2]:
columns = ['trn_acc', 'tst_acc', 'oos_trn_pred', 'oos_tst_pred', 'model_path']
result_file_dir = "../results/out_of_sample/"
#result_file_dir = "../results/out_of_sample_mnist_bak/"
_, _, grid_params, _ = OutOfSampleReprClf()()
df = params_to_dataframe(grid_params,
                         columns=columns,
                         result_file_dir=result_file_dir,
                         verbose=1, logging_level=0)
df['arch'] = df['model'].apply(get_arch)
df['model name'] = [get_model_name(d['model'], d['eps']) for _, d in df.iterrows()]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
2022-06-29 10:01:24 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar10wo0-cwl2-64-cifar10wo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:24 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar10wo0-cwl2-64-cifar10wo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 4.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}


Missing ../results/out_of_sample/cwl2-128-calcedrepr-cifar10wo0-cwl2-64-cifar10wo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-cifar10wo0-cwl2-64-cifar10wo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-4.0-0.01-trades6ce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl


2022-06-29 10:01:27 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:27 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 4.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}


problem with ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-4.0-0.01-trades6ce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-8.0-0.01-trades6ce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl


2022-06-29 10:01:27 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 8.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:27 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 32.0, 'learning_rate': 0.01, 'model': 'trades6ce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:27 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo2-cwl2-128-aug10-imgnet100wo2-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'lea

...

2022-06-29 10:01:28 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar10wo9-cwl2-64-cifar10wo9-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:28 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar100coarsewo0-cwl2-64-cifar100coarsewo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:28 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar100coarsewo4-cwl2-64-cifar100coarsewo4-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP

Missing ../results/out_of_sample/cwl2-128-calcedrepr-cifar100coarsewo0-cwl2-64-cifar100coarsewo0-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-cifar100coarsewo4-cwl2-64-cifar100coarsewo4-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-cifar100coarsewo9-cwl2-64-cifar100coarsewo9-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl


2022-06-29 10:01:28 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-cifar100coarsewo9-cwl2-64-cifar100coarsewo9-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:28 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo0-cwl2-128-aug10-imgnet100wo0-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
2022-06-29 10:01:29 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01

Missing ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo0-cwl2-128-aug10-imgnet100wo0-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo1-cwl2-128-aug10-imgnet100wo1-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl
Missing ../results/out_of_sample/cwl2-128-calcedrepr-aug10-imgnet100wo2-cwl2-128-aug10-imgnet100wo2-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-advce-vtor2-VggMLP-0.0-2-adam-0-0.0.pkl


2022-06-29 10:01:29 ERROR    Error with {'attack': 'cwl2', 'batch_size': 128, 'dataset': 'calcedrepr-aug10-imgnet100wo2-cwl2-128-aug10-imgnet100wo2-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl', 'epochs': 70, 'eps': 2.0, 'learning_rate': 0.01, 'model': 'advce-vtor2-VggMLP', 'momentum': 0.0, 'norm': '2', 'optimizer': 'adam', 'random_seed': 0, 'weight_decay': 0.0}
[Parallel(n_jobs=1)]: Done 419 out of 419 | elapsed:   41.0s finished


In [3]:
ncg_acc = []
for _, di in tqdm(df.iterrows()):
    if isinstance(di['oos_trn_pred'], Iterable):
        net_pred = np.concatenate((di['oos_trn_pred'], di['oos_tst_pred']))
        res = joblib.load(f"./out_of_sample/{di['dataset']}.pkl")
        nn_pred = np.concatenate((
            res[f'miss_trn_knn_y_{di["norm"]}'].reshape(-1),
            res[f'miss_tst_knn_y_{di["norm"]}'].reshape(-1)))
        ncg_acc.append((net_pred == nn_pred).mean())
    else:
        ncg_acc.append(-1)
df['ncg acc'] = ncg_acc

0it [00:00, ?it/s]

In [4]:
datasets, ds_names, archs = [], [], []
accepted_models = ['natural', 'mixup', 'AT(0.5)', 'AT(1)', 'AT(2)', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)']

datasets += [f'calcedreprold-mnistwo{i}-cwl2-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl' for i in range(10)]
ds_names += [f"MNIST-wo{i}" for i in range(10)]
archs += ["LargeMLP"] * 10
#accepted_models += ['natural', 'AT(2)', 'TRADES(2)', 'DBS6($\lambda$=1)', 'ellipsoid', 'sub-Voronoi']

datasets += [f"calcedrepr-cifar10wo{i}-cwl2-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl" for i in [0, 4, 9]]
ds_names += [f"CIFAR10-wo{i}" for i in [0, 4, 9]]
archs += ["LargeMLP"] * 3
##accepted_models = ['natural', 'AT(2)', 'TRADES(2)', 'TRADES(4)', 'DBS6($\lambda$=1)', 'ellipsoid', 'sub-Voronoi']

datasets += [f"calcedrepr-cifar100coarsewo{i}-cwl2-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl" for i in [0, 4, 9]]
ds_names += [f"CIFAR100-wo{i}" for i in [0, 4, 9]]
archs += ["LargeMLP"] * 3

datasets += [f'calcedrepr-aug10-imgnet100wo{i}-cwl2-128-aug10-imgnet100wo{i}-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl' for i in range(3)]
ds_names += [f"ImgNet100-wo{i}" for i in range(3)]
archs += ["LargeMLPv4"] * 3
##accepted_models = ['natural', 'AT', 'TRADES', 'DBS6($\lambda$=1)']

#columns = ['natural', 'AT(2)', 'TRADES(2)', 'DBS']

dfs = []
all_data = {}
for ds_name, arch in zip(datasets, archs):
    td = df[df['dataset'].str.contains(ds_name) & df['arch'].str.contains(arch)]
    td = td[['model name', 'arch', 'trn_acc', 'tst_acc', 'ncg acc', 'eps', 'learning_rate', 'batch_size', 'dataset', 'random_seed']]
    short_ds_name = ds_names[datasets.index(ds_name)]
    
    ddfs = []
    used_model_names = []
    for i, (_, d) in enumerate(td.iterrows()):
        if d['model name'] in ['TRADES', 'AT']:
            if d['eps'] >= 1:
                d['model name'] = f"{d['model name']}({int(d['eps'])})"
            else:
                d['model name'] = f"{d['model name']}({float(d['eps'])})"
        if "imgnet" in ds_name:
            if d['model name'] == "TRADES(2)":
                if d['learning_rate'] != 0.01:
                    continue
            elif d['learning_rate'] != 0.1:
                continue
        if "cifar" in ds_name:
            if d['learning_rate'] != 0.01:
                continue
            if d['arch'] == 'LargeMLPv3':
                continue
            
        if d['model name'] not in accepted_models:
            continue
        if d['model name'] in used_model_names:
            continue
        data = []
            
        all_data[(short_ds_name, d['model name'])] = (d['trn_acc'], d['tst_acc'], d['ncg acc'])
        used_model_names.append(d['model name'])

In [5]:
#accepted_models = ['natural', 'mixup', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)', 'AT(2)']
accepted_models = ['natural', 'mixup', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)']
#accepted_models = ['natural', 'TRADES(2)', 'TRADES(4)', 'TRADES(8)', 'AT(0.5)']
for model in accepted_models:
    temp = []
    #for ds_name in [f'MNIST-wo{i}' for i in range(10)]:
    #for ds_name in [f"CIFAR10-wo{i}" for i in [0, 4, 9]]:
    #for ds_name in [f"CIFAR100-wo{i}" for i in [0, 4, 9]]:
    for ds_name in [f"ImgNet100-wo{i}" for i in range(3)]:
        temp.append(all_data[(ds_name, model)][2])
    print(np.mean(temp), np.std(temp))

0.11975308641975309 0.008054077249530917
0.13827160493827162 0.011826103718348685
0.16074074074074074 0.014299686158587909
0.13111111111111112 0.010066082523772602
0.13753086419753088 0.005811655454785095


In [23]:
df[df['dataset'].str.contains("mnistwo0") & df['arch'].str.contains("MLP")]

,attack,batch_size,dataset,epochs,eps,learning_rate,model,momentum,norm,optimizer,random_seed,weight_decay,trn_acc,tst_acc,oos_trn_pred,oos_tst_pred,model_path,arch,model name,ncg acc
18,cwl2,256,calcedreprold-mnistwo0-cwl2-128-mnistwo0-70-1....,70,1.0,0.01,ce-vtor2-LargeMLP,0.9,2,sgd,0,0.0,1.000000,0.993792,"[5, 7, 1, 5, 7, 5, 7, 1, 1, 5, 1, 5, 5, 5, 1, ...","[5, 1, 5, 5, 1, 1, 8, 5, 5, 1, 1, 5, 1, 1, 5, ...",./models/out_of_sample/pgd-256-calcedrepr-mnis...,LargeMLP,natural,0.276112
28,cwl2,256,calcedreprold-mnistwo0-cwl2-128-mnistwo0-70-1....,70,2.0,0.01,trades6ce-vtor2-LargeMLP,0.9,2,sgd,0,0.0,0.999186,0.994568,"[5, 5, 5, 5, 7, 5, 7, 5, 7, 7, 7, 5, 5, 5, 1, ...","[5, 5, 5, 5, 1, 7, 8, 5, 5, 1, 1, 5, 5, 1, 5, ...",./models/out_of_sample/pgd-256-calcedrepr-mnis...,LargeMLP,TRADES,0.389975
29,cwl2,256,calcedreprold-mnistwo0-cwl2-128-mnistwo0-70-1....,70,4.0,0.01,trades6ce-vtor2-LargeMLP,0.9,2,sgd,0,0.0,0.997670,0.994568,"[5, 5, 5, 5, 8, 5, 7, 1, 7, 5, 1, 5, 5, 5, 5, ...","[5, 5, 5, 5, 1, 7, 5, 5, 5, 1, 1, 5, 5, 8, 5, ...",./models/out_of_sample/pgd-256-calcedrepr-mnis...,LargeMLP,TRADES,0.485876
30,cwl2,256,calcedreprold-mnistwo0-cwl2-128-mnistwo0-70-1....,70,8.0,0.01,trades6ce-vtor2-LargeMLP,0.9,2,sgd,0,0.0,0.996043,0.994457,"[5, 5, 5, 5, 4, 5, 7, 5, 7, 7, 7, 5, 5, 5, 5, ...","[5, 8, 8, 5, 7, 7, 5, 5, 5, 8, 1, 5, 5, 8, 6, ...",./models/out_of_sample/pgd-256-calcedrepr-mnis...,LargeMLP,TRADES,0.547733
58,cwl2,256,calcedreprold-mnistwo0-cwl2-128-mnistwo0-70-1....,70,2.0,0.01,advce-vtor2-LargeMLP,0.9,2,sgd,0,0.0,0.999982,0.994900,"[5, 5, 5, 5, 7, 5, 7, 1, 7, 5, 1, 5, 5, 5, 7, ...","[5, 1, 5, 5, 1, 1, 5, 5, 5, 1, 1, 5, 5, 1, 5, ...",./models/out_of_sample/pgd-256-calcedrepr-mnis...,LargeMLP,AT,0.324207


In [6]:
ddff = pd.DataFrame.from_dict(all_data, orient="index", columns=['train acc.', 'test acc.', 'NCG acc.'])
ddff.index = pd.MultiIndex.from_tuples(ddff.index)
ddff = ddff.stack(0).unstack(1)[accepted_models]
text = ddff.to_latex(float_format="%.2f", multirow=True)
text = text.replace("DBS6(\$\\textbackslash lambda\$=1)", "ball")
text = text.replace("llrrrrr", "llccccc")
print(text)

\begin{tabular}{llccccc}
\toprule
          &          &  natural &  mixup &  TRADES(2) &  TRADES(4) &  TRADES(8) \\
\midrule
\multirow{3}{*}{CIFAR10-wo0} & train acc. &     1.00 &   1.00 &       1.00 &       1.00 &       1.00 \\
          & test acc. &     0.89 &   0.90 &       0.89 &       0.90 &       0.90 \\
          & NCG acc. &     0.80 &   0.78 &       0.81 &       0.83 &       0.83 \\
\cline{1-7}
\multirow{3}{*}{CIFAR10-wo4} & train acc. &     1.00 &   1.00 &       1.00 &       1.00 &       1.00 \\
          & test acc. &     0.88 &   0.88 &       0.88 &       0.89 &       0.88 \\
          & NCG acc. &     0.82 &   0.78 &       0.82 &       0.85 &       0.85 \\
\cline{1-7}
\multirow{3}{*}{CIFAR10-wo9} & train acc. &     1.00 &   1.00 &       1.00 &       1.00 &       1.00 \\
          & test acc. &     0.88 &   0.88 &       0.88 &       0.89 &       0.89 \\
          & NCG acc. &     0.84 &   0.82 &       0.83 &       0.88 &       0.87 \\
\cline{1-7}
\multirow{3}{*}{CIFAR100-

### t-test (rob vs. nat)

In [7]:
results = {}
for model_name in accepted_models[1:]:
    tt = ddff['natural'] - ddff[model_name]
    for ds_name in ds_names:
        key = (ds_name.split('-')[0], model_name)
        results[key] = results.setdefault(key, 0) + (tt.loc[(ds_name, 'NCG acc.')] < 0)
#pd.DataFrame.from_dict(results)
results

{('MNIST', 'mixup'): 9,
 ('CIFAR10', 'mixup'): 0,
 ('CIFAR100', 'mixup'): 2,
 ('ImgNet100', 'mixup'): 3,
 ('MNIST', 'TRADES(2)'): 9,
 ('CIFAR10', 'TRADES(2)'): 2,
 ('CIFAR100', 'TRADES(2)'): 3,
 ('ImgNet100', 'TRADES(2)'): 3,
 ('MNIST', 'TRADES(4)'): 10,
 ('CIFAR10', 'TRADES(4)'): 3,
 ('CIFAR100', 'TRADES(4)'): 3,
 ('ImgNet100', 'TRADES(4)'): 3,
 ('MNIST', 'TRADES(8)'): 10,
 ('CIFAR10', 'TRADES(8)'): 3,
 ('CIFAR100', 'TRADES(8)'): 3,
 ('ImgNet100', 'TRADES(8)'): 3}

### small NCG table

In [12]:
#small_df = ddff.loc[[("MNIST-wo0", "NCG acc."), ("MNIST-wo9", "NCG acc."), ("CIFAR10-wo0", "NCG acc."), ("CIFAR100-wo0", "NCG acc."), ("ImgNet100-wo0", "NCG acc.")]]
small_df = ddff.loc[[("MNIST-wo0", "NCG acc."), ("MNIST-wo9", "NCG acc."), ("CIFAR10-wo0", "NCG acc."), ("CIFAR100-wo0", "NCG acc.")]]
small_df.index = small_df.index.droplevel(1)
text = small_df.transpose().to_latex(float_format="%.2f")
text = text.replace("MNIST-wo", "M-")
text = text.replace("CIFAR10-wo", "C10-")
text = text.replace("CIFAR100-wo", "C100-")
text = text.replace("ImgNet100-wo", "I-")
text = text.replace("0.", ".")
print(text)

\begin{tabular}{lrrrr}
\toprule
{} &  M-0 &  M-9 &  C10-0 &  C100-0 \\
\midrule
natural   &       .28 &       .61 &         .80 &          .63 \\
mixup     &       .35 &       .77 &          NaN &           NaN \\
TRADES(2) &       .39 &       .71 &         .81 &          .69 \\
TRADES(4) &       .49 &       .74 &         .83 &          .68 \\
TRADES(8) &       .55 &       .80 &         .83 &          .68 \\
AT(2)     &       .32 &       .71 &         .39 &          .09 \\
\bottomrule
\end{tabular}



In [39]:
small_df.to_json()

'{"natural":{"MNIST-wo0":0.2761118354,"MNIST-wo9":0.6632653061,"CIFAR10-wo0":0.7951666667,"CIFAR100-wo0":0.6326666667,"ImgNet100-wo0":0.0992592593},"AT(2)":{"MNIST-wo0":0.3221787629,"MNIST-wo9":0.7009198045,"CIFAR10-wo0":0.3866666667,"CIFAR100-wo0":0.0903333333,"ImgNet100-wo0":0.0155555556},"TRADES(2)":{"MNIST-wo0":0.3905548312,"MNIST-wo9":0.7053751078,"CIFAR10-wo0":0.8111666667,"CIFAR100-wo0":0.6923333333,"ImgNet100-wo0":0.122962963},"TRADES(4)":{"MNIST-wo0":0.4482109228,"MNIST-wo9":0.7298074159,"CIFAR10-wo0":0.8318333333,"CIFAR100-wo0":0.6786666667,"ImgNet100-wo0":0.1377777778},"TRADES(8)":{"MNIST-wo0":0.5769955092,"MNIST-wo9":0.7786720322,"CIFAR10-wo0":0.8258333333,"CIFAR100-wo0":0.6843333333,"ImgNet100-wo0":0.1214814815}}'

## in-distribution NCG

In [7]:
from lolip.models.torch_utils import archs
from lolip.variables import auto_var

def get_preds(model, dset, batch_size=64, device="cuda"):
    model.eval().to(device)
    if isinstance(dset, np.ndarray):
        dset = torch.utils.data.TensorDataset(
            torch.from_numpy(dset).float(),
            torch.ones(len(dset))
        )
    loader = torch.utils.data.DataLoader(dset, batch_size=batch_size, shuffle=False, num_workers=24)

    ret = []
    for (x, _) in tqdm(loader, desc="[get_preds]"):
        with torch.no_grad():
            output = F.softmax(model(x.to(device)), dim=1)
        ret.append(output.cpu().numpy())
    del loader
    return np.concatenate(ret, axis=0)

In [30]:
inncg_acc = {}

In [31]:

model_paths = []
#model_paths += [(
#    (f"MNIST-wo{i}", "TRADES(2)"),
#    f"calcedrepr-mnistwo{i}-cwl2-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
#    f"../models/out_of_sample/mnist_bak/pgd-256-calcedrepr-mnistwo{i}-pgd-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl-70-1.0-0.01-ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt",
#) for i in range(10)]
#model_paths += [(
#    (f"MNIST-wo{i}", "natural"),
#    f"calcedrepr-mnistwo{i}-cwl2-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl",
#    f"../models/out_of_sample/mnist_bak/pgd-256-calcedrepr-mnistwo{i}-pgd-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt",
#) for i in range(10)]
model_paths += [(
    (f"CIFAR10-wo{i}", "natural"),
    f"calcedrepr-cifar10wo{i}-cwl2-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar10wo{i}-pgd-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-1.0-0.01-ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 4, 9]]
model_paths += [(
    (f"CIFAR10-wo{i}", "TRADES(2)"),
    f"calcedrepr-cifar10wo{i}-cwl2-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar10wo{i}-pgd-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 4, 9]]
model_paths += [(
    (f"CIFAR100-wo{i}", "natural"),
    f"calcedrepr-cifar100coarsewo{i}-cwl2-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar100coarsewo{i}-pgd-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-1.0-0.01-ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 4, 9]]
model_paths += [(
    (f"CIFAR100-wo{i}", "TRADES(2)"),
    f"calcedrepr-cifar100coarsewo{i}-cwl2-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar100coarsewo{i}-pgd-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 4, 9]]
model_paths += [(
    (f"ImgNet100-wo{i}", "natural"),
    f"calcedrepr-aug10-imgnet100wo{i}-cwl2-128-aug10-imgnet100wo{i}-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar100coarsewo{i}-pgd-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-1.0-0.01-ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 1, 2]]
model_paths += [(
    (f"ImgNet100-wo{i}", "TRADES(2)"),
    f"calcedrepr-aug10-imgnet100wo{i}-cwl2-128-aug10-imgnet100wo{i}-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl",
    f'../models/out_of_sample/pgd-128-calcedrepr-cifar100coarsewo{i}-pgd-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl-70-2.0-0.01-trades6ce-vtor2-LargeMLP-0.9-2-sgd-0-0.0-ep0070.pt',
) for i in [0, 1, 2]]



for key, dataset, model_path in tqdm(model_paths):
    if key in inncg_acc:
        continue
    _, trny, tstX, tsty, _ = auto_var.get_var_with_argument("dataset", dataset)
    arch_name = model_path.split("-")[-7]
    model = getattr(archs, arch_name)(n_classes=len(np.unique(tsty)), n_features=(tstX.shape[1], ))
    model.load_state_dict(torch.load(model_path)['model_state_dict'])

    pred = get_preds(model, tstX).argmax(1)

    res = joblib.load(f"./out_of_sample/tst_k5/{dataset}.pkl")
    nn_pred = trny[res['l2_ind'][:, 0].reshape(-1)]
    inncg_acc[key] = [(pred == tsty).mean(), (pred == nn_pred).mean()]

  0%|          | 0/12 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/141 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/141 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/141 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/141 [00:00<?, ?it/s]

...

[get_preds]:   0%|          | 0/149 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/149 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/149 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/149 [00:00<?, ?it/s]

[get_preds]:   0%|          | 0/149 [00:00<?, ?it/s]

In [32]:
inncg_acc

{('CIFAR10-wo0', 'natural'): [0.8945555555555555, 0.9523333333333334],
 ('CIFAR10-wo4', 'natural'): [0.8803333333333333, 0.949],
 ('CIFAR10-wo9', 'natural'): [0.8801111111111111, 0.9542222222222222],
 ('CIFAR10-wo0', 'TRADES(2)'): [0.8913333333333333, 0.9563333333333334],
 ('CIFAR10-wo4', 'TRADES(2)'): [0.8782222222222222, 0.9521111111111111],
 ('CIFAR10-wo9', 'TRADES(2)'): [0.8803333333333333, 0.9552222222222222],
 ('CIFAR100-wo0', 'natural'): [0.7176842105263158, 0.8208421052631579],
 ('CIFAR100-wo4', 'natural'): [0.7246315789473684, 0.818],
 ('CIFAR100-wo9', 'natural'): [0.7031578947368421, 0.8088421052631579],
 ('CIFAR100-wo0', 'TRADES(2)'): [0.7327368421052631, 0.8517894736842105],
 ('CIFAR100-wo4', 'TRADES(2)'): [0.7318947368421053, 0.8483157894736842],
 ('CIFAR100-wo9', 'TRADES(2)'): [0.7152631578947368, 0.8431578947368421]}

In [36]:
res = joblib.load("../results/oos_repr/cwl2-128-aug10-imgnet100-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl")

In [41]:
res["tst_acc"]

0.5266

In [46]:
datasets = []
datasets += [f'calcedrepr-mnistwo{i}-cwl2-128-mnistwo{i}-70-1.0-0.01-ce-vtor2-CNN002-0.9-2-sgd-0-0.0.pkl' for i in [0, 4, 9]]
#datasets += [f"calcedrepr-cifar10wo{i}-cwl2-64-cifar10wo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl" for i in [0, 4, 9]]
#datasets += [f"calcedrepr-cifar100coarsewo{i}-cwl2-64-cifar100coarsewo{i}-70-1.0-0.01-ce-vtor2-WRN_40_10-0.0-2-adam-0-0.0.pkl" for i in [0, 4, 9]]
#datasets += [f'calcedrepr-aug10-imgnet100wo{i}-cwl2-128-aug10-imgnet100wo{i}-70-1.0-0.01-ce-vtor2-ResNet50Norm01-0.0-2-adam-0-0.0.pkl' for i in [0, 1, 2]]

nn_accs = []
for dataset in tqdm(datasets):
    _, trny, tstX, tsty, _ = auto_var.get_var_with_argument("dataset", dataset)

    res = joblib.load(f"./out_of_sample/tst_k5/{dataset}.pkl")
    nn_pred = trny[res['l2_ind'][:, 0].reshape(-1)]
    nn_accs.append((nn_pred == tsty).mean())
nn_accs

  0%|          | 0/3 [00:00<?, ?it/s]

[0.9876940133037694, 0.9894655134176092, 0.9903236569903237]

In [43]:
nn_pred

array([24, 19, 36, ..., 61, 25,  0])

In [45]:
tsty

array([ 0,  0,  0, ..., 98, 98, 98])